# Lecture 4: Python Review 2

## Key Concepts Review

### Binary Numbers and Float Precision

Computers store numbers in binary (base 2), which can cause precision issues with decimal numbers:

```python
# Float precision issues
print(0.1 + 0.2)  # Output: 0.30000000000000004 (not exactly 0.3!)
print(0.1 + 0.2 == 0.3)  # Output: False

# Better way to compare floats
def close_enough(a, b, tolerance=1e-9):
    return abs(a - b) < tolerance

print(close_enough(0.1 + 0.2, 0.3))  # Output: True
```

### Objects and Types
Everything in Python (except keywords) is an object with a type:

```python
# Scalar types (single values)
concentration = 0.5      # float
num_atoms = 6           # int
element = "H"           # string
is_stable = True        # boolean

# Non-scalar types (collections)
concentrations = [0.1, 0.2, 0.3]     # list
compound_data = {"name": "water", "formula": "H2O"}  # dictionary

# Check types
print(type(concentration))  # <class 'float'>
print(type(concentrations)) # <class 'list'>

# Type comparison
print(isinstance(concentration, float)) # True
print(isinstance(concentration, list)) # False
```


### Useful Operators

```python
# Comparison operators that handle precision
import math
math.isclose(0.1 + 0.2, 0.3)  # Better than == for floats

# Membership operators
"H" in ["H", "O", "N"]  # True
"formula" in {"name": "water", "formula": "H2O"}  # True
```

## Exercise 1: Concentration Precision Checker

**Objective:** Understand float precision issues when working with chemical concentrations.

**Problem:** In analytical chemistry, you often need to compare measured concentrations, but computer precision can cause issues. Write a function that safely compares concentrations and handles rounding errors.

**Your Task:** Create a function called `concentrations_equal()` that:

- Takes two concentration values and an optional tolerance (default 1e-6)
- Returns True if the concentrations are "close enough" considering float precision

Also create a function `round_to_significant_figures()` that rounds a number to a given number of significant figures

**Hints:**
- Use abs(a - b) < tolerance for comparison
- For significant figures, you might need to use logarithms: import math then math.log10()
- Think about how to handle very small numbers (like 0.00123)


In [1]:

def concentrations_equal(conc1: float, conc2: float, tolerance : float = 1E-6) -> bool:
    return abs(conc1-conc2) < tolerance


In [2]:
# using string conversions and then interpreting as float
def round_to_significant_figures_str(value: float, precision: int) -> float:
    return float(f"{value:.{precision}g}")
    
# using log10 but a bit convoluted?!
import math
def round_to_significant_figures_log(value: float, precision: int) -> float:
    exponent = math.log10(value)
    normalized = value/(10**round(exponent))
    return round(normalized, ndigits=precision-1)*(10**round(exponent))

### Test Cases: Concentration comparison


In [3]:
print(concentrations_equal(0.1 + 0.2, 0.3))           # Should be True
print(concentrations_equal(0.1001, 0.1002, 1e-3))     # Should be True  
print(concentrations_equal(0.1001, 0.1002, 1e-5))     # Should be False

True
True
False


### Test Cases: Rounding

In [4]:
print(round_to_significant_figures_str(0.123456, 3))       # Should be 0.123
print(round_to_significant_figures_str(1234.56, 3))        # Should be 1230.0
print(round_to_significant_figures_str(0.00123456, 3))     # Should be 0.00123

print(round_to_significant_figures_log(0.123456, 3))       # Should be 0.123
print(round_to_significant_figures_log(1234.56, 3))        # Should be 1230.0
print(round_to_significant_figures_log(0.00123456, 3))     # Should be 0.00123

0.123
1230.0
0.00123
0.123
1230.0
0.00123


## Exercise 2: Reaction Optimization Calculator

**Objective:** Use optimization thinking to find the best reaction conditions while working with different data types.

**Problem:** You're optimizing a chemical reaction by testing different temperatures and catalyst amounts. Write a program that finds the optimal conditions from experimental data.

**Your Task:** Create functions that:

- Store experimental data as a list of dictionaries
- Find the conditions that give maximum yield
- Find all conditions that give yield within a certain percentage of the maximum
- Calculate the cost-effectiveness (yield per unit cost) for each condition

**Data Structure:**

In [5]:
# Each experiment is a dictionary
experimental_data = [
    {"temperature": 25.0, "catalyst": 0.1, "yield": 45.2, "cost": 10.5},
    {"temperature": 50.0, "catalyst": 0.1, "yield": 67.8, "cost": 12.0},
    {"temperature": 75.0, "catalyst": 0.1, "yield": 82.3, "cost": 15.5},
    {"temperature": 100.0, "catalyst": 0.1, "yield": 78.9, "cost": 20.0},
    {"temperature": 50.0, "catalyst": 0.2, "yield": 75.4, "cost": 18.0},
    {"temperature": 75.0, "catalyst": 0.2, "yield": 89.1, "cost": 22.5}
]

**Functions to Write:** 

In [6]:
from typing import List, Dict
# Backwards-compatible -- for Python 3.8 and higher can just use list, dict below, no import

def find_max_yield_condition(data: List[Dict[str, float]]) -> Dict[str, float]:
    # Return the dictionary with highest yield
    return sorted(data, key=lambda x: x['yield'])[-1]

In [7]:

def find_conditions_near_optimum(data: List[Dict[str, float]], percentage: int = 10) -> List[Dict[str, float]]:
    # Return all conditions within percentage% of max yield   
    best_yield_exp = find_max_yield_condition(data)
    return [exp for exp in data if (abs(exp['yield']-best_yield_exp['yield']) < percentage)]
    
# [f(x) for x in iterable if condition]

In [8]:
def calculate_cost_effectiveness(data: List[Dict[str, float]]) -> List[Dict[str, float]]:
    # Add "efficiency" key to each dictionary (yield/cost)
    # Return the modified data
    
    for entry in data:
        entry['efficiency'] = entry['yield'] / entry['cost']
    
    return data
        
    

In [9]:

def optimize_by_criteria(data: List[Dict[str, float]], criteria: str = "yield") -> List[Dict[str, float]]:
    # Return best condition based on "yield", "cost", or "efficiency"
    return sorted(data, key=lambda entry: entry[criteria])[-1]


### Test case:

In [10]:
# Basic Optimization
best_yield = find_max_yield_condition(experimental_data)
print(f"Best yield: {best_yield}")
# Expected: Should return the condition with 89.1% yield

Best yield: {'temperature': 75.0, 'catalyst': 0.2, 'yield': 89.1, 'cost': 22.5}


In [11]:
near_optimum = find_conditions_near_optimum(experimental_data, 15)
print(f"Conditions within 15% of optimum: {len(near_optimum)} found")
# Expected: Should find multiple conditions within 15% of 89.1%

Conditions within 15% of optimum: 4 found


In [12]:
data_with_efficiency = calculate_cost_effectiveness(experimental_data)
best_efficiency = optimize_by_criteria(data_with_efficiency, "efficiency")
print(f"Most cost-effective: {best_efficiency}")

Most cost-effective: {'temperature': 50.0, 'catalyst': 0.1, 'yield': 67.8, 'cost': 12.0, 'efficiency': 5.6499999999999995}


## Exercise 3: Reaction Simulator Class

**Objective:** Create a reaction simulator that models the concentration of the reagents and products as a reaction progresses. 

**Problem:** Build a ChemicalReaction class that simulates simple chemical reactions, tracks reactant consumption, and handles errors when invalid data is provided.

**Your Task:** Complete the class below:

In [13]:
import math
from typing import Dict
from collections import defaultdict
from itertools import chain

class ChemicalReaction:
    def __init__(self, reactants: Dict[str, int], products: Dict[str, int], reaction_rate: int = 1.0):
        """
        Initialize a chemical reaction
        reactants: dict like {"H2": 2, "O2": 1}
        products: dict like {"H2O": 2}
        the terms reagent and reactant are used here interchangeably but reactant is strictly more correct
        The integer values for reactants and products indicate the reaction stoichiometry
        reaction_rate: float (how fast the reaction proceeds, in one reaction given by reactants and products per second)
        """

        # To be initialised
        self.reactants = reactants
        self.products = products
        self.reaction_rate = reaction_rate

        self.reaction_vessel_content: Dict[str, float] = defaultdict(float)
    
    def add_reactant_amount(self, compound: str, amount: float):
        """Add amount of reactant to the reaction vessel. What happens if the compound added is not part of the reagents or products?"""
        if compound not in set(chain(self.reactants.keys(), self.products.keys())):
            raise ValueError("Reactant not part of reaction.")

        self.reaction_vessel_content[compound] += amount
            
            
    def run_reaction(self, time_seconds: float = 1.0):
        """Run reaction for given time and update reactant amounts. What happens if not enough of one compound is left?"""
        reaction_amount = self.reaction_rate * time_seconds

        # limitation given by either 1) the previous maximum_reaction_amount for the last molecule, 
        # 2) the reaction_amount 3) availability in the vessel
        maximum_reaction_amount = float("inf")
        for molecule, amount in self.reactants.items():
            maximum_reaction_amount = min([maximum_reaction_amount, 
                                           reaction_amount, 
                                           self.reaction_vessel_content[molecule]/amount * reaction_amount])
            
        for compound in self.reactants.keys():
            self.reaction_vessel_content[compound] -= self.reactants[compound] * maximum_reaction_amount

        for compound in self.products.keys():
            self.reaction_vessel_content[compound] += self.products[compound] * maximum_reaction_amount
        
    
    def get_reaction_status(self):
        """Return current content of the reaction vessel as a dictionary."""
        return self.reaction_vessel_content
        

### Test cases

In [14]:
water_reaction = ChemicalReaction(
    reactants={"H2": 2, "O2": 1},
    products={"H2O": 2},
    reaction_rate=0.5
)


In [15]:
water_reaction.get_reaction_status() 

defaultdict(float, {})

In [16]:
## Test case 1: General function of the simulator

# Add reactants
water_reaction.add_reactant_amount("H2", 10.0)
water_reaction.add_reactant_amount("O2", 3.0)

# Check initial state of reaction vessel
reaction_status = water_reaction.get_reaction_status() 
print(reaction_status)
assert reaction_status['H2'] == 10.0
assert reaction_status['O2'] == 3.0
assert reaction_status['H2O'] == 0

# Run reaction
water_reaction.run_reaction(2.0)  # Run for 2 timepoints

# Check content of reaction vessel again
reaction_status = water_reaction.get_reaction_status() 
print(reaction_status)
assert reaction_status['H2'] == 8.0
assert reaction_status['O2'] == 2.0
assert reaction_status['H2O'] == 2.0

defaultdict(<class 'float'>, {'H2': 10.0, 'O2': 3.0})
defaultdict(<class 'float'>, {'H2': 8.0, 'O2': 2.0, 'H2O': 2.0})


In [17]:
## Test case 2: Test error handling when adding compound not in reaction
water_reaction.add_reactant_amount("N2", 10.0)

ValueError: Reactant not part of reaction.

In [18]:
## Test case 3: Let reaction run to completion
water_reaction.add_reactant_amount("H2", 2.0)
water_reaction.run_reaction(4.0)

# Check content of reaction vessel again
reaction_status = water_reaction.get_reaction_status() 
print(reaction_status)
assert reaction_status['H2'] == 6.0
assert reaction_status['O2'] == 0.0
assert reaction_status['H2O'] == 6.0

defaultdict(<class 'float'>, {'H2': 6.0, 'O2': 0.0, 'H2O': 6.0})
